In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from tf_imports import *
from data_generator import *
from tf_models import *
import time
from utils import limit_mem
tf.compat.v1.enable_eager_execution()

/export/home/gmooers/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/export/home/gmooers/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/export/home/gmooers/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/export/home/gmooer

FileNotFoundError: [Errno 2] No such file or directory: '/export/home/gmooers/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/keras/api/_v1/keras/callbacks/hyai_hybi.pkl'

In [2]:
DATADIR = 'Preprocessed_Data/7_Years_Spaced/'

train_gen = DataGenerator(
    data_dir=DATADIR, 
    feature_fn='full_physics_essentials_train_month01_shuffle_features.nc',
    target_fn='full_physics_essentials_train_month01_shuffle_targets.nc',
    batch_size=512,
    norm_fn='full_physics_essentials_train_month01_norm.nc',
    fsub='feature_means', 
    fdiv='feature_stds', 
    tmult='target_conv',
    shuffle=True,
)

valid_gen = DataGenerator(
    data_dir=DATADIR, 
    feature_fn='full_physics_essentials_valid_month02_features.nc',
    target_fn='full_physics_essentials_valid_month02_targets.nc',
    batch_size=512,
    norm_fn='full_physics_essentials_train_month01_norm.nc',  # SAME NORMALIZATION FILE!
    fsub='feature_means', 
    fdiv='feature_stds', 
    tmult='target_conv',
    shuffle=False,
)

Generator will have 378224640 samples in 738720 batches
Features have shape 64; targets have shape 65
Generator will have 47278080 samples in 92340 batches
Features have shape 64; targets have shape 65


In [3]:
#inp = Input(shape=(train_gen.feature_shape,))
inp = tf.keras.Input(shape=(train_gen.feature_shape,))
out = tf.keras.layers.Dense(train_gen.target_shape)(inp)

In [4]:
model = tf.keras.experimental.LinearModel(inputs=inp, outputs=out)
model.compile(optimizer='sgd', loss='mse')

In [5]:
model.summary()

Model: "linear_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 65)                4225      
Total params: 4,225
Trainable params: 4,225
Non-trainable params: 0
_________________________________________________________________


In [8]:
callbacks = [tf.keras.callbacks.TensorBoard(log_dir='my_log_dir/Linear_Regression', histogram_freq=0)]

In [9]:
h = model.fit_generator(
    train_gen.return_generator(False, False), 
    train_gen.n_batches,
    epochs=1,
    validation_data=valid_gen.return_generator(False, False),
    validation_steps=valid_gen.n_batches, workers =16, max_queue_size =50, callbacks=callbacks
)

AttributeError: 'threadsafe_iter' object has no attribute 'shape'

In [ ]:
#other visual tensorboard information
from keras.utils import plot_model
import pydot
from keras.utils.vis_utils import model_to_dot
keras.utils.vis_utils.pydot = pydot
plot_model(model, to_file='my_log_dir/Linear_Regression/model.png')
plot_model(model, show_shapes=True, to_file='my_log_dir/Linear_Regression/models.png')



#Save the model
model.save('Models/Linear_Regression.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model
print('It Finished')



import matplotlib.pyplot as plt

t1 = time.time()
total = t1-t0
total = total/(60.0*60.0)

hdict1 = h.history
train_loss_values1 = hdict1['loss']
valid_loss_values1 = hdict1['val_loss']
epochs1 = range(1, len(train_loss_values1) + 1)
plt.plot(epochs1, train_loss_values1, 'bo', label='Train loss')
plt.plot(epochs1, valid_loss_values1, 'b', label='Valid loss')
plt.title('Training and validation loss - SPCAM5; time:'+str(total)[:8]+' hours')
plt.xlabel('Epochs', fontsize = 20)
plt.ylabel('Loss', fontsize = 20)
#plt.yscale('log')
plt.legend()
plt.savefig('Figures/Linear_Regression.png')